In [ ]:
# 安装 Python 及常用库在 VS Code 中打开你要工作的文件夹 (File -> Open Folder)。

# 新建并打开一个 Jupyter Notebook (.ipynb文件) 或使用 Python Interactive Window
# pip install pandas numpy matplotlib lets-plot pingouin skimpy

# 步骤2：导入库和设置
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pingouin as pg
from lets_plot import *

LetsPlot.setup_html(no_js=True)

# 可选：设置图表样式
plt.style.use(
    "https://raw.githubusercontent.com/aeturrell/core_python/main/plot_style.txt"
)
"""
Part 1.1 平均地表温度随时间的变化
数据获取与加载：

步骤3：下载数据

前往 NASA’s Goddard Institute for Space Studies 的 相应页面 下载‘Northern Hemisphere-mean monthly, seasonal, and annual means’的CSV文件（文件名默认NH.Ts+dSST.csv）。
将该文件放在你的工作目录中（例如当前文件夹）
"""
"""
Part 1.1 平均地表温度随时间的变化
数据获取与加载：

步骤3：下载数据

前往 NASA’s Goddard Institute for Space Studies 的 相应页面 下载‘Northern Hemisphere-mean monthly, seasonal, and annual means’的CSV文件（文件名默认NH.Ts+dSST.csv）
"""
# 加载数据集
df = pd.read_csv(
    "NH.Ts+dSST.csv",
    skiprows=1,
    na_values="***",
)
df.head()
df.info()
# 步骤5：将Year设为索引，并绘制时序图
df = df.set_index("Year")
month = "Jan"
fig, ax = plt.subplots()
ax.axhline(0, color="orange")
ax.annotate("1951–1980 average", xy=(0.66, -0.2), xycoords=("figure fraction", "data"))
df[month].plot(ax=ax)
ax.set_title(f"Average temperature anomaly in {month} (1880—{df.index.max()})")
ax.set_ylabel("Temperature anomaly (°C)")
ax.set_xlabel("Year")
plt.show()

# 步骤6：尝试绘制季节和年度平均值
# 季节列：DJF, MAM, JJA, SON
seasons = ["DJF","MAM","JJA","SON"]

for season in seasons:
    fig, ax = plt.subplots()
    ax.axhline(0, color="orange")
    ax.annotate("1951–1980 average", xy=(0.66, -0.2), xycoords=("figure fraction", "data"))
    df[season].plot(ax=ax)
    ax.set_title(f"Average temperature anomaly in {season} (1880—{df.index.max()})")
    ax.set_ylabel("Temperature anomaly (°C)")
    ax.set_xlabel("Year")
    plt.show()

# 年平均（J-D列）
month = "J-D"
fig, ax = plt.subplots()
ax.axhline(0, color="orange")
ax.annotate("1951–1980 average", xy=(0.68, -0.2), xycoords=("figure fraction", "data"))
df[month].plot(ax=ax)
ax.set_title(
    f"Average annual temperature anomaly in the northern hemisphere (1880—{df.index.max()})"
)
ax.set_ylabel("Annual temperature anomalies (°C)")
ax.set_xlabel("Year")
plt.show()
#Part 1.2 温度随时间的变化分布和变异性
# 步骤7：创建分类变量并绘制直方图
df["Period"] = pd.cut(
    df.index,
    bins=[1921, 1950, 1980, 2010],
    labels=["1921—1950", "1951—1980", "1981—2010"],
    ordered=True,
)

list_of_months = ["Jun","Jul","Aug"]
fig, axes = plt.subplots(ncols=3, figsize=(9,4), sharex=True, sharey=True)
for ax, period in zip(axes, df["Period"].dropna().unique()):
    df.loc[df["Period"] == period, list_of_months].stack().hist(ax=ax)
    ax.set_title(period)
plt.suptitle("Histogram of temperature anomalies")
axes[1].set_xlabel("Summer temperature distribution")
plt.tight_layout()
plt.show()
# 步骤8：计算分位数并比较冷热异常值所占比例
# 1951-1980期间所有月份数据
temp_all_months = df.loc[(df.index>=1951)&(df.index<=1980),"Jan":"Dec"].stack().reset_index().rename(columns={"level_1":"month",0:"values"})
quantiles = [0.3,0.7]
list_of_percentiles = np.quantile(temp_all_months["values"], q=quantiles)
print(f"30%冷阈值: {list_of_percentiles[0]}")
print(f"70%热阈值: {list_of_percentiles[1]}")
# 在1981–2010期间计算热天比例
temp_all_months_81_10 = df.loc[(df.index>=1981)&(df.index<=2010),"Jan":"Dec"].stack().reset_index().rename(columns={"level_1":"month",0:"values"})
proportion_over_q70 = (temp_all_months_81_10["values"]>list_of_percentiles[1]).mean()
print(f"1981-2010年高于{list_of_percentiles[1]}的比例为: {proportion_over_q70*100:.2f}%")
# 步骤9：计算各季节在不同时期的平均值和方差
temp_seasons = df.loc[:, "DJF":"SON"].stack().reset_index().rename(columns={"level_1":"Season",0:"Values"})
temp_seasons["Period"] = pd.cut(
    temp_seasons["Year"],
    bins=[1921,1950,1980,2010],
    labels=["1921—1950","1951—1980","1981—2010"],
    ordered=True
)

grp_mean_var = temp_seasons.groupby(["Season","Period"])["Values"].agg([np.mean,np.var])
grp_mean_var
#Part 1.3 碳排放与气候的关系
# 步骤10：下载并导入 CO₂ 数据 将CO₂数据 (1_CO2-data.csv) 下载至 data文件夹
df_co2 = pd.read_csv("data/1_CO2-data.csv")
df_co2.head()
# 步骤11：绘制 CO₂ 时间序列图
# 绘制 interpolated 和 trend 随年份变化图表
#  Year, Month 列，从1960年开始
df_co2_1960 = df_co2[df_co2["Year"]>=1960]

fig, ax = plt.subplots()
ax.plot(df_co2_1960["Year"] + df_co2_1960["Month"]/12, df_co2_1960["Interpolated"], label="Interpolated")
ax.plot(df_co2_1960["Year"] + df_co2_1960["Month"]/12, df_co2_1960["Trend"], label="Trend")
ax.set_title("CO₂ Levels Over Time (from 1960)")
ax.set_xlabel("Year")
ax.set_ylabel("CO₂ concentration (ppm)")
ax.legend()
plt.show()
# 步骤12：合并 CO₂ 与温度数据并计算相关系数
df_co2_june = df_co2[df_co2["Month"]==6]
df_temp_co2 = pd.merge(df_co2_june, df, on="Year")
df_temp_co2[["Jun","Trend"]].corr(method="pearson")
# 绘制散点图：
(
    ggplot(df_temp_co2, aes(x="Jun", y="Trend"))
    + geom_point(color="black", size=3)
    + labs(
        title="Scatterplot of temperature anomalies vs carbon dioxide emissions",
        x="Temperature anomaly (°C)",
        y="Carbon dioxide levels (trend)"
    )
)
# 在 Notebook 中，你需要：
#
# 将上面所有代码分块运行。
# 在相应的 Markdown 单元格中回答问题。
# 对比图表并给出分析解释。
# 在完成后，你可以将 .ipynb 文件提交至你的 GitHub 仓库（git add ., git commit -m "完成练习", git push）以保存和展示你的工作成果